In [ ]:
import cv2
import glob
import numpy as np
import math

In [ ]:
from keras.layers import Input, merge, Concatenate, Dense, Dropout, Conv2D, Add, Dot, Lambda, Conv2DTranspose, Dot, Activation, Reshape, BatchNormalization, UpSampling2D, AveragePooling2D, GlobalAveragePooling2D, Multiply, LeakyReLU, Flatten, MaxPool2D 
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers import PReLU, LeakyReLU

from keras import optimizers
from keras.models import Model
from keras.models import load_model
import keras.backend as K
import tensorflow as tf


In [ ]:
def build_encoder(input_shape, name='encoder', encode_channels=[8,16, 32, 64]):
    input_layer = Input(shape=input_shape)
    
    
    for index, channel in enumerate(encode_channels):
        
        shortcut = Conv2D(channel, 3, padding='same', trainable=False)(input_layer if index == 0 else encoder_block)
        encoder_block = BatchNormalization()(input_layer if index == 0 else encoder_block)
        encoder_block = LeakyReLU()(encoder_block)
        encoder_block = Conv2D(channel, 3, padding='same')(encoder_block)    
        
        encoder_block = BatchNormalization()(encoder_block)
        encoder_block = LeakyReLU()(encoder_block)
        encoder_block = Conv2D(channel, 3, padding='same')(encoder_block)
        
        
        encoder_block = Add()([encoder_block, shortcut])
        
        encoder_block = Conv2D(channel, 3, padding='same', strides=2)(encoder_block)
        encoder_block = LeakyReLU()(encoder_block)
        
    
    output_layer = encoder_block
    return Model(input_layer, output_layer, name=name)
encoder = build_encoder((128,128,3), encode_channels=[16, 32])
encoder.summary()

In [ ]:
_, x, y, z = encoder.output_shape
compression = (x*y*z)/((128**2)*3) * 128
print(f'Encoder reduces dimensionality to {compression}% of original size.')

In [ ]:
def build_decoder(input_shape, name='decoder', decode_channels = [64, 32, 16, 8], use_upsampling=False):
    input_layer = Input(shape=input_shape)
    
    
    for index, channel in enumerate(decode_channels):
        shortcut = Conv2D(channel, 3, padding='same', trainable=False)(input_layer if index == 0 else decoder_block)
        
        decoder_block = BatchNormalization()(input_layer if index == 0 else decoder_block)
        decoder_block = LeakyReLU()(decoder_block)
        decoder_block = Conv2D(channel, 3, padding='same')(decoder_block)
        
        decoder_block = BatchNormalization()(decoder_block)
        decoder_block = LeakyReLU()(decoder_block)
        decoder_block = Conv2D(channel, 3, padding='same')(decoder_block)
        
        decoder_block = Add()([decoder_block, shortcut])
        
        if use_upsampling:
            decoder_block = UpSampling2D(interpolation='nearest')(decoder_block)
            decoder_block = Conv2D(channel, 3, padding='same')(decoder_block)
            

        decoder_block = Conv2DTranspose(channel, 3, padding='same', strides=2)(decoder_block)


    
    output_layer = Conv2D(3, 3, padding='same', activation='sigmoid')(decoder_block)
    return Model(input_layer, output_layer, name=name)
decoder = build_decoder((32, 32, 32), decode_channels=[32, 16], use_upsampling=False)
decoder.summary()

In [ ]:
def build_autoencoder(encoder, decoder, name='autoencoder'):
    input_img = Input(shape=(128,128,3))
    latent_representation = encoder(input_img)
    decoded_img = decoder(latent_representation)
    autoencoder = Model(input_img, decoded_img, name=name)
    return autoencoder
autoencoder = build_autoencoder(encoder, decoder)
autoencoder.summary()

In [ ]:
def data_loader(file_path):
    img_paths = glob.glob(file_path + '/*')
    data = []
    for img_path in img_paths:
        
        data.append(cv2.imread(img_path) / 255.0)
    return np.array(data)

In [ ]:
X = data_loader('./cleaned-data/')

In [ ]:
# grap only the first 1600 images
subset_X = X[:1600] 

In [ ]:
import matplotlib.pyplot as plt
def plot_history(history):
    plt.plot(history.history['acc'])
    plt.title("Accuracy")
    plt.show()
    plt.plot(history.history['loss'])
    plt.title("Loss")
    plt.show()

In [ ]:
import keras
# custom metric to avoid the binary_accuracy rounding in keras
def image_closeness(y_pred, y_true):
    return K.mean(1 - K.abs(y_pred - y_true), axis=-1)
keras.metrics.image_closeness = image_closeness

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse', metrics=[image_closeness, 'accuracy'])


In [ ]:
history = autoencoder.fit(subset_X, subset_X, epochs=100, batch_size=32, shuffle=True)
plot_history(history)

In [ ]:
hist.append(history)


In [ ]:
hist = []
for i in range(1, 6):
    history = autoencoder.fit(subset_X, subset_X, epochs=1000, batch_size=32, shuffle=True)
    hist.append(history)
    autoencoder.save('./autoencoder-v3-models/autoencoder-v3-' + str(i) + 'k-epochs.h5')
    print(i, 'k epochs~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    plot_history(history)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    

In [ ]:
accs = []
losses = []
for i, history_segment in enumerate(hist):
    
    losses.extend(history_segment.history['loss'])
    accs.extend(history_segment.history['acc'])


In [ ]:
plt.plot(accs)
plt.title("Accuracy")
plt.savefig('./autoencoder-v3-models-tanh/accuracy')
plt.show()
plt.plot(losses)
plt.title("Loss")
plt.savefig('./autoencoder-v3-models-tanh/loss')
plt.show()


In [ ]:
autoencoder = load_model('./autoencoder-v3-models/autoencoder-v3-2k-epochs.h5')

In [ ]:
testimgs = data_loader('./testxclean/')

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
# figure(num=None, figsize=(10, 10), dpi=200, facecolor='w', edgecolor='k')
def rgb_imshow(img, name):
#     figure(num=None, dpi=200, facecolor='w', edgecolor='k')
    plt.xticks([])
    plt.yticks([])
#     plt.title(name, loc='left')
    plt.imshow(cv2.cvtColor((img).astype(np.float32), cv2.COLOR_BGR2RGB))
#     plt.savefig('./testimgs/' + name + '.jpg', bbox_inches='tight')
    
def show_infer_image(img_index, model, imgname):
    
    figure(num=None, dpi=300, facecolor='w', edgecolor='k')

    img_input = testimgs[0] 
    scaled_img = cv2.resize((img_input*255).astype(np.uint8), (86,86))
    decoded_img = model.predict(np.array([img_input]), steps=None)[0]
    ax = plt.subplot(2, 4, 1)
    ax.set_ylabel('Input Image')
    rgb_imshow(img_input, None)
    ax = plt.subplot(2, 4, 5)
    ax.set_ylabel('Decoded Image')
    rgb_imshow(decoded_img, None)
    
    img_input = testimgs[1] 
    scaled_img = cv2.resize((img_input*255).astype(np.uint8), (86,86))
    decoded_img = model.predict(np.array([img_input]), steps=None)[0]
    plt.subplot(2, 4, 2)
    rgb_imshow(img_input, None)
    plt.subplot(2, 4, 6)
    rgb_imshow(decoded_img, None)
    
    img_input = testimgs[2] 
    scaled_img = cv2.resize((img_input*255).astype(np.uint8), (86,86))
    decoded_img = model.predict(np.array([img_input]), steps=None)[0]
    plt.subplot(2, 4, 3)
    rgb_imshow(img_input, None)
    plt.subplot(2, 4, 7)
    rgb_imshow(decoded_img, None)
    
    img_input = testimgs[3] 
    scaled_img = cv2.resize((img_input*255).astype(np.uint8), (86,86))
    decoded_img = model.predict(np.array([img_input]), steps=None)[0]
    plt.subplot(2, 4, 4)
    rgb_imshow(img_input, None)
    plt.subplot(2, 4, 8)
    rgb_imshow(decoded_img, None)
    



    plt.tight_layout()
    plt.savefig('./autoencoder-v3-models/results.png', bbox_inches='tight')
    plt.show()

    
# for i in range(0, 4):
#     rgb_imshow(testimgs[i], 'Input Image')
show_infer_image(i, autoencoder, 'Residual Decoded')
    